# yfinanceを使用した日本株式データ取得テストノートブック

このノートブックでは、`sumalize.py`で実際に使用している値の取得方法を解説しながらテストできます。

## 目的

- `sumalize.py`の`get_stock_data`関数で取得しているデータ項目を順を追って確認
- 各データ項目の取得方法
- テスト用のティッカーシンボルで実際のデータ取得

---

## テスト対象の設定


In [35]:
import yfinance as yf
import pandas as pd

# ティッカーシンボルを指定（例: 7203.T = トヨタ自動車、1828.T = サンヨーホームズ）
# sumalize.pyのformat_ticker関数と同じ形式: "コード.T"
ticker_symbol = "1828.T"
ticker = yf.Ticker(ticker_symbol)

print(f"📊 テスト対象: {ticker_symbol}")
print("=" * 60)

📊 テスト対象: 1828.T


---

## 1. 基本情報の取得

`ticker.info`から基本情報を取得します。

### 取得項目
- **会社名** (`longName`, `shortName`)
- **業種** (`industry`)
- **セクター** (`sector`)
- **郵便番号** (`zip`) - 後で都道府県に変換
- **現在の株価** (`currentPrice`)
- **時価総額** (`marketCap`)


In [36]:
info = ticker.info

print("📋 【基本情報】")
print("-" * 60)
print(f"会社名（longName）: {info.get('longName', 'N/A')}")
print(f"会社名（shortName）: {info.get('shortName', 'N/A')}")
print(f"業種（industry）: {info.get('industry', 'N/A')}")
print(f"セクター（sector）: {info.get('sector', 'N/A')}")
print(f"郵便番号（zip）: {info.get('zip', 'N/A')}")
print(f"現在の株価（currentPrice）: {info.get('currentPrice', 'N/A')}")
if info.get('marketCap'):
    print(f"時価総額（marketCap）: {info.get('marketCap'):,.0f}")
else:
    print("時価総額: N/A")
print("=" * 60)

📋 【基本情報】
------------------------------------------------------------
会社名（longName）: Tanabe Engineering Corporation
会社名（shortName）: TANABE ENGINEERING CORP
業種（industry）: Engineering & Construction
セクター（sector）: Industrials
郵便番号（zip）: 942-0032
現在の株価（currentPrice）: 2465.0
時価総額（marketCap）: 25,880,158,208


---

## 2. 市場データ（時価総額、PBR、PER）

市場で評価される指標を取得します。

### 取得項目
- **時価総額** (`marketCap`) - 株価 × 発行済み株式数
- **PBR** (`priceToBook`) - 株価純資産倍率
- **PER(会予)** (`forwardPE`) - 予想PER
- **PER直近** (`trailingPE`) - 過去12ヶ月の実績PER


In [37]:
if 'info' not in locals():
    info = ticker.info

print("📈 【市場データ】")
print("-" * 60)
if info.get('marketCap'):
    print(f"時価総額（marketCap）: {info.get('marketCap'):,.0f}")
else:
    print("時価総額: N/A")
print(f"PBR（priceToBook）: {info.get('priceToBook', 'N/A')}")

# PER(会予) - 予想PER
forward_pe = info.get('forwardPE', None)
print(f"PER(会予)（forwardPE）: {forward_pe}")

# PER直近 - 過去12ヶ月の実績PER
trailing_pe = info.get('trailingPE', None)
print(f"PER直近（trailingPE）: {trailing_pe}")

# sumalize.pyのcalculate_recent_pe関数と同じ計算方法
# PER = 現在の株価 / trailingEps（過去12ヶ月のEPS）
current_price = info.get('currentPrice') or info.get('regularMarketPrice')
trailing_eps = info.get('trailingEps')
if current_price and trailing_eps and trailing_eps != 0:
    calculated_pe = current_price / trailing_eps
    print(f"\n💡 計算したPER直近（currentPrice / trailingEps）: {calculated_pe:.2f}")
    if trailing_pe:
        diff = abs(calculated_pe - trailing_pe)
        print(f"   → trailingPEとの差: {diff:.4f}")
print("=" * 60)

📈 【市場データ】
------------------------------------------------------------
時価総額（marketCap）: 25,880,158,208
PBR（priceToBook）: 1.0150675
PER(会予)（forwardPE）: None
PER直近（trailingPE）: 9.195017

💡 計算したPER直近（currentPrice / trailingEps）: 9.20
   → trailingPEとの差: 0.0000


---

## 3. 配当関連データ

株主への配当に関する情報を取得します。

### 取得項目
- **配当方向性** (`payoutRatio`) - 配当性向（利益に対する配当の割合）
- **配当利回り** (`trailingAnnualDividendYield`) - 過去12ヶ月の配当利回り
- **配当金履歴** (`ticker.dividends`) - 過去の配当額の推移



In [38]:
if 'info' not in locals():
    info = ticker.info

print("💰 【配当関連データ】")
print("-" * 60)

# 配当方向性（配当性向）- sumalize.pyでは「配当方向性」として保存
payout_ratio = info.get('payoutRatio', None)
print(f"配当方向性（payoutRatio）: {payout_ratio}")
if payout_ratio:
    print(f"  → 配当性向: {payout_ratio * 100:.2f}%")

# 配当利回り（過去12ヶ月）
dividend_yield = info.get('trailingAnnualDividendYield', None)
print(f"\n配当利回り（trailingAnnualDividendYield）: {dividend_yield}")
if dividend_yield:
    print(f"  → 配当利回り: {dividend_yield * 100:.2f}%")

# 配当金履歴（実際の配当額の推移）
dividends = ticker.dividends
if not dividends.empty:
    print(f"\n📅 直近の配当金履歴（最新5件）:")
    print(dividends.tail(5))
    latest_dividend = dividends.iloc[-1]
    print(f"\n最新配当額: {latest_dividend}円")
else:
    print("\n配当金履歴: データなし")
print("=" * 60)

💰 【配当関連データ】
------------------------------------------------------------
配当方向性（payoutRatio）: 0.3245
  → 配当性向: 32.45%

配当利回り（trailingAnnualDividendYield）: 0.035568275
  → 配当利回り: 3.56%

📅 直近の配当金履歴（最新5件）:
Date
2021-03-30 00:00:00+09:00    25.0
2022-03-30 00:00:00+09:00    33.0
2023-03-30 00:00:00+09:00    40.0
2024-03-28 00:00:00+09:00    50.0
2025-03-28 00:00:00+09:00    87.0
Name: Dividends, dtype: float64

最新配当額: 87.0円


---

## 4. EPS関連データ

1株あたりの利益（Earnings Per Share）を取得します。

### 取得項目
- **EPS(過去12ヶ月)** (`trailingEps`) - 過去12ヶ月の実績EPS
- **EPS(予想)** (`forwardEps`) - 予想EPS



In [39]:
if 'info' not in locals():
    info = ticker.info

print("📊 【EPS関連データ】")
print("-" * 60)

# EPS(過去12ヶ月) - 実績EPS
trailing_eps = info.get('trailingEps', None)
print(f"EPS(過去12ヶ月)（trailingEps）: {trailing_eps}")

# EPS(予想) - 予想EPS
forward_eps = info.get('forwardEps', None)
print(f"EPS(予想)（forwardEps）: {forward_eps}")
print("=" * 60)

📊 【EPS関連データ】
------------------------------------------------------------
EPS(過去12ヶ月)（trailingEps）: 268.08
EPS(予想)（forwardEps）: None


---

## 5. 収益性指標

企業の収益性を評価する指標を取得します。

### 取得項目
- **ROE** (`returnOnEquity`) - 自己資本利益率
- **営業利益率** (`operatingMargins`) - 売上に対する営業利益の割合
- **純利益率** (`profitMargins`) - 売上に対する純利益の割合


In [40]:
if 'info' not in locals():
    info = ticker.info

print("📈 【収益性指標】")
print("-" * 60)

# ROE（自己資本利益率）
roe = info.get('returnOnEquity', None)
print(f"ROE（returnOnEquity）: {roe}")
if roe:
    print(f"  → ROE: {roe * 100:.2f}%")

# 営業利益率
operating_margins = info.get('operatingMargins', None)
print(f"\n営業利益率（operatingMargins）: {operating_margins}")
if operating_margins:
    print(f"  → 営業利益率: {operating_margins * 100:.2f}%")

# 純利益率
profit_margins = info.get('profitMargins', None)
print(f"\n純利益率（profitMargins）: {profit_margins}")
if profit_margins:
    print(f"  → 純利益率: {profit_margins * 100:.2f}%")
print("=" * 60)


📈 【収益性指標】
------------------------------------------------------------
ROE（returnOnEquity）: 0.11486
  → ROE: 11.49%

営業利益率（operatingMargins）: 0.09564
  → 営業利益率: 9.56%

純利益率（profitMargins）: 0.055549998
  → 純利益率: 5.55%


---

## 6. 財務諸表データ（損益計算書）

損益計算書（Income Statement）から主要な財務データを取得します。

### 取得項目
- **売上高** (`Total Revenue`)
- **営業利益** (`Operating Income`)
- **当期純利益** (`Net Income`)


In [41]:
print("📑 【財務諸表データ（損益計算書）】")
print("-" * 60)

try:
    financials = ticker.financials
    
    if not financials.empty:
        print(f"📊 財務諸表の構造:")
        print(f"  決算期数: {len(financials.columns)}")
        print(f"  項目数: {len(financials.index)}")
        print(f"\n📅 決算期一覧（最新から）:")
        for i, col in enumerate(financials.columns[:5]):  # 最新5期のみ表示
            print(f"  {i+1}. {col}")
        
        # 最新決算期（最初の列）を取得
        latest_period = financials.columns[0]
        print(f"\n最新決算期: {latest_period}")
        
        # sumalize.pyと同じ方法でデータを取得
        def safe_get_financial_item(df, item_name, fallback_items=None):
            """sumalize.pyのsafe_get_financial_data関数と同じロジック"""
            if item_name in df.index:
                value = df.loc[item_name, latest_period]
                return value if pd.notna(value) else None
            
            if fallback_items:
                for fallback in fallback_items:
                    if fallback in df.index:
                        value = df.loc[fallback, latest_period]
                        return value if pd.notna(value) else None
            return None
        
        # 売上高
        total_revenue = safe_get_financial_item(financials, "Total Revenue")
        if total_revenue:
            print(f"\n💰 売上高（Total Revenue）: {total_revenue:,.0f}")
        else:
            print("\n売上高: N/A")
        
        # 営業利益
        operating_income = safe_get_financial_item(financials, "Operating Income")
        if operating_income:
            print(f"💵 営業利益（Operating Income）: {operating_income:,.0f}")
        else:
            print("営業利益: N/A")
        
        # 当期純利益
        net_income = safe_get_financial_item(financials, "Net Income")
        if net_income:
            print(f"💴 当期純利益（Net Income）: {net_income:,.0f}")
        else:
            print("当期純利益: N/A")
        
        # 財務諸表の一部を表示
        print(f"\n📋 財務諸表データ（主要項目、最新決算期）:")
        key_items = ["Total Revenue", "Operating Income", "Net Income", "Total Operating Expenses"]
        for item in key_items:
            if item in financials.index:
                value = financials.loc[item, latest_period]
                if pd.notna(value):
                    print(f"  • {item}: {value:,.0f}")
    else:
        print("❌ 財務諸表データが取得できませんでした")
        
except Exception as e:
    print(f"❌ 財務諸表取得エラー: {e}")

print("=" * 60)


📑 【財務諸表データ（損益計算書）】
------------------------------------------------------------
📊 財務諸表の構造:
  決算期数: 4
  項目数: 46

📅 決算期一覧（最新から）:
  1. 2025-03-31 00:00:00
  2. 2024-03-31 00:00:00
  3. 2023-03-31 00:00:00
  4. 2022-03-31 00:00:00

最新決算期: 2025-03-31 00:00:00

💰 売上高（Total Revenue）: 50,832,166,000
💵 営業利益（Operating Income）: 3,837,548,000
💴 当期純利益（Net Income）: 2,593,138,000

📋 財務諸表データ（主要項目、最新決算期）:
  • Total Revenue: 50,832,166,000
  • Operating Income: 3,837,548,000
  • Net Income: 2,593,138,000


---

## 7. バランスシートデータ

貸借対照表（Balance Sheet）から財務健全性に関するデータを取得します。

### 取得項目
- **負債** (`Total Liabilities Net Minority Interest` / `Total Liab`)
- **流動負債** (`Current Liabilities` / `Total Current Liabilities`)
- **流動資産** (`Current Assets` / `Total Current Assets`)
- **総負債** (`Total Debt`)
- **現金及び現金同等物** (`Cash And Cash Equivalents`)
- **投資有価証券** (`Available For Sale Securities` / `Short Term Investments`)
- **自己資本** (`Stockholders Equity` / `Total Stockholder Equity`)
- **総資産** (`Total Assets`)

In [42]:
print("📊 【バランスシートデータ】")
print("-" * 60)

try:
    balance_sheet = ticker.balance_sheet
    
    if not balance_sheet.empty:
        print(f"📋 バランスシートの構造:")
        print(f"  決算期数: {len(balance_sheet.columns)}")
        print(f"  項目数: {len(balance_sheet.index)}")
        
        # 最新決算期（最初の列）を取得
        latest_period = balance_sheet.columns[0]
        print(f"\n最新決算期: {latest_period}")
        
        # sumalize.pyと同じ方法でデータを取得（フォールバック付き）
        def safe_get_balance_item(df, item_name, fallback_items=None):
            """sumalize.pyのsafe_get_financial_data関数と同じロジック"""
            if item_name in df.index:
                value = df.loc[item_name, latest_period]
                return value if pd.notna(value) else None
            
            if fallback_items:
                for fallback in fallback_items:
                    if fallback in df.index:
                        value = df.loc[fallback, latest_period]
                        return value if pd.notna(value) else None
            return None
        
        # 負債（フォールバック付き）
        total_liabilities = safe_get_balance_item(
            balance_sheet, 
            "Total Liabilities Net Minority Interest",
            fallback_items=["Total Liab"]
        )
        if total_liabilities:
            print(f"\n💳 負債（Total Liabilities）: {total_liabilities:,.0f}")
        else:
            print("\n負債: N/A")
        
        # 流動負債（フォールバック付き）
        current_liabilities = safe_get_balance_item(
            balance_sheet,
            "Current Liabilities",
            fallback_items=["Total Current Liabilities"]
        )
        if current_liabilities:
            print(f"💳 流動負債（Current Liabilities）: {current_liabilities:,.0f}")
        else:
            print("流動負債: N/A")
        
        # 流動資産（フォールバック付き）
        current_assets = safe_get_balance_item(
            balance_sheet,
            "Current Assets",
            fallback_items=["Total Current Assets"]
        )
        if current_assets:
            print(f"💵 流動資産（Current Assets）: {current_assets:,.0f}")
        else:
            print("流動資産: N/A")
        
        # 総負債
        total_debt = safe_get_balance_item(balance_sheet, "Total Debt")
        if total_debt:
            print(f"💳 総負債（Total Debt）: {total_debt:,.0f}")
        else:
            print("総負債: N/A")
        
        # 現金及び現金同等物（フォールバック付き）
        cash_and_equivalents = safe_get_balance_item(
            balance_sheet,
            "Cash And Cash Equivalents",
            fallback_items=["Cash Cash Equivalents And Short Term Investments"]
        )
        if cash_and_equivalents:
            print(f"💰 現金及び現金同等物: {cash_and_equivalents:,.0f}")
        else:
            print("現金及び現金同等物: N/A")
        
        # 投資有価証券（フォールバック付き）
        investments = safe_get_balance_item(
            balance_sheet,
            "Available For Sale Securities",
            fallback_items=["Short Term Investments", "Investmentin Financial Assets"]
        )
        if investments:
            print(f"📈 投資有価証券: {investments:,.0f}")
        else:
            print("投資有価証券: N/A")
        
        # 自己資本（フォールバック付き）
        total_equity = safe_get_balance_item(
            balance_sheet,
            "Stockholders Equity",
            fallback_items=["Total Stockholder Equity"]
        )
        if total_equity:
            print(f"💼 自己資本（Stockholders Equity）: {total_equity:,.0f}")
        else:
            print("自己資本: N/A")
        
        # 総資産
        total_assets = safe_get_balance_item(balance_sheet, "Total Assets")
        if total_assets:
            print(f"🏢 総資産（Total Assets）: {total_assets:,.0f}")
        else:
            print("総資産: N/A")
        
    else:
        print("❌ バランスシートデータが取得できませんでした")
        
except Exception as e:
    print(f"❌ バランスシート取得エラー: {e}")

print("=" * 60)


📊 【バランスシートデータ】
------------------------------------------------------------
📋 バランスシートの構造:
  決算期数: 4
  項目数: 62

最新決算期: 2025-03-31 00:00:00

💳 負債（Total Liabilities）: 20,988,127,000
💳 流動負債（Current Liabilities）: 19,091,627,000
💵 流動資産（Current Assets）: 35,010,887,000
💳 総負債（Total Debt）: 1,426,641,000
💰 現金及び現金同等物: 10,849,611,000
📈 投資有価証券: 330,657,000
💼 自己資本（Stockholders Equity）: 24,980,783,000
🏢 総資産（Total Assets）: 45,976,310,000


---

## 8. 計算項目

取得したデータから、追加の指標を計算します。

### 計算項目
1. **自己資本比率** = 自己資本 / 総資産
2. **ネットキャッシュ** = 流動資産 + (投資有価証券 × 0.7) - 負債
3. **ネットキャッシュ比率** = ネットキャッシュ / 時価総額

In [47]:
print("🧮 【計算項目】")
print("-" * 60)

# 前のセルで取得したデータを使用（変数が定義されている場合）
try:
    # 自己資本比率の計算
    if 'total_equity' in locals() and 'total_assets' in locals():
        if total_equity and total_assets:
            equity_ratio = total_equity / total_assets
            print(f"📊 自己資本比率 = 自己資本 / 総資産")
            print(f"  = {total_equity:,.0f} / {total_assets:,.0f}")
            print(f"  = {equity_ratio:.4f} ({equity_ratio * 100:.2f}%)")
        else:
            print("❌ 自己資本比率: 計算不可（データ不足）")
    else:
        print("⚠️  自己資本比率: 前のセル（セル7）を実行してください")
    
    # ネットキャッシュの計算（sumalize.pyのcalculate_net_cash関数と同じロジック）
    if 'current_assets' in locals() and 'total_liabilities' in locals():
        if current_assets is not None and total_liabilities is not None:
            # 投資有価証券は70%で評価（保守的な見積もり）
            inv_value = (investments * 0.7) if 'investments' in locals() and investments is not None else 0
            net_cash = current_assets + inv_value - total_liabilities
            
            print(f"\n💰 ネットキャッシュ = 流動資産 + (投資有価証券 × 0.7) - 負債")
            if 'investments' in locals() and investments:
                print(f"  = {current_assets:,.0f} + ({investments:,.0f} × 0.7) - {total_liabilities:,.0f}")
                print(f"  = {current_assets:,.0f} + {inv_value:,.0f} - {total_liabilities:,.0f}")
            else:
                print(f"  = {current_assets:,.0f} + 0 - {total_liabilities:,.0f}")
            print(f"  = {net_cash:,.0f}")
            
            # ネットキャッシュ比率の計算
            if 'info' not in locals():
                info = ticker.info
            market_cap = info.get('marketCap')
            if net_cash and market_cap:
                net_cash_ratio = net_cash / market_cap
                print(f"\n📈 ネットキャッシュ比率 = ネットキャッシュ / 時価総額")
                print(f"  = {net_cash:,.0f} / {market_cap:,.0f}")
                print(f"  = {net_cash_ratio:.4f} ({net_cash_ratio * 100:.2f}%)")
            else:
                print("\n❌ ネットキャッシュ比率: 計算不可（時価総額データ不足）")
        else:
            print("\n❌ ネットキャッシュ: 計算不可（データ不足）")
    else:
        print("\n⚠️  ネットキャッシュ: 前のセル（セル7）を実行してください")
    
except NameError as e:
    print(f"❌ 計算エラー: 必要なデータが取得できていません。前のセルを実行してください。")
    print(f"  エラー詳細: {e}")

print("=" * 60)


🧮 【計算項目】
------------------------------------------------------------
📊 自己資本比率 = 自己資本 / 総資産
  = 24,980,783,000 / 45,976,310,000
  = 0.5433 (54.33%)

💰 ネットキャッシュ = 流動資産 + (投資有価証券 × 0.7) - 負債
  = 35,010,887,000 + (330,657,000 × 0.7) - 20,988,127,000
  = 35,010,887,000 + 231,459,900 - 20,988,127,000
  = 14,254,219,900

📈 ネットキャッシュ比率 = ネットキャッシュ / 時価総額
  = 14,254,219,900 / 25,880,158,208
  = 0.5508 (55.08%)


---

## 9. 決算月の取得

バランスシートの最新決算期から決算月を取得します。

### 取得方法
- バランスシートの最新決算期（最初の列）から日付を抽出
- datetimeオブジェクトの場合は`strftime`でフォーマット
- 文字列の場合は時間部分を削除


In [44]:
print("📅 【決算月の取得】")
print("-" * 60)

try:
    balance_sheet = ticker.balance_sheet
    
    if not balance_sheet.empty:
        cols = balance_sheet.columns.tolist()
        if cols:
            # 最新決算期から日付部分のみ抽出（sumalize.pyと同じロジック）
            latest_period = cols[0]
            
            if hasattr(latest_period, "strftime"):
                # datetimeオブジェクトの場合、日付部分のみ取得
                settlement_period = latest_period.strftime("%Y-%m-%d")
            else:
                # 文字列の場合、時間部分を削除
                settlement_period = str(latest_period).split(" ")[0]
            
            print(f"最新決算期（生データ）: {latest_period}")
            print(f"決算月（フォーマット後）: {settlement_period}")
        else:
            print("❌ 決算期データがありません")
    else:
        print("❌ 決算月: データ取得不可")
        
except Exception as e:
    print(f"❌ 決算月取得エラー: {e}")

print("=" * 60)


📅 【決算月の取得】
------------------------------------------------------------
最新決算期（生データ）: 2025-03-31 00:00:00
決算月（フォーマット後）: 2025-03-31
